# Generate Embeddings
This notebook demonstrates how to generate embeddings from tweets containing a ME Bank hashtag or mention.

## Update PYTHHONPATH
Until this notebook has a setup.py, insert parent directory onto PYTHONPATH

In [1]:
import sys

sys.path.append("../")

## Scrape Twitter
Scrape Twitter for any hashtag or mention for the ME Bank account (#mebank or @mebank). This will create a DataFrame with date, content, content_type, user, and a url for each tweet.

In [2]:
from RG271.scrapers.twitter import TwitterScraper

scraper = TwitterScraper("mebank", min_date="2020-07-27")
df = scraper.scrape()

Here are some of the tweets

In [3]:
df.head(10)

,date,content,content_type,user,url
0,2021-07-25 09:22:13+00:00,@mebank Making beautiful banking and helping A...,twitter/mention,sandybeech4,https://twitter.com/sandybeech4/status/1419226...
1,2021-07-24 01:09:02+00:00,So @mebank made me come all the way to police ...,twitter/mention,TheDadhiwala,https://twitter.com/TheDadhiwala/status/141873...
2,2021-07-23 13:21:12+00:00,@mebank So... do you have a timeframe when is ...,twitter/mention,spikeuk1982,https://twitter.com/spikeuk1982/status/1418561...
3,2021-07-22 23:39:22+00:00,@mebank How do you justify taking 3 weeks to g...,twitter/mention,heathwilder,https://twitter.com/heathwilder/status/1418355...
4,2021-07-22 10:15:06+00:00,"Congrats to Ella Van De Velde Fidock, Register...",twitter/mention,qnmuofficial,https://twitter.com/qnmuofficial/status/141815...
5,2021-07-22 06:58:58+00:00,"@mebank hey Andy, I spoke to one already and h...",twitter/mention,EllisonAmy,https://twitter.com/EllisonAmy/status/14181032...
6,2021-07-22 05:01:02+00:00,@EllisonAmy @mebank I switched to @bankaust ab...,twitter/mention,lukedastoli,https://twitter.com/lukedastoli/status/1418073...
7,2021-07-22 04:47:47+00:00,@ChrisBinney @mebank The worstttttttttt and wh...,twitter/mention,EllisonAmy,https://twitter.com/EllisonAmy/status/14180702...
8,2021-07-22 04:26:52+00:00,@EllisonAmy @mebank Go with @UBank 2.34%,twitter/mention,rezntoumos,https://twitter.com/rezntoumos/status/14180649...
9,2021-07-22 04:18:03+00:00,@EllisonAmy @mebank How bad is that app! So slow,twitter/mention,ChrisBinney,https://twitter.com/ChrisBinney/status/1418062...


## Download and load a pretrained model
Use an already trained Sentence Transformer model to embed sentences.

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("paraphrase-mpnet-base-v2")

Generate sentence embeddings.

In [5]:
embeddings = model.encode(df["content"].values.tolist())